In [4]:
import sqlite3
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import chromadb
import numpy as np
import zipfile
import io
import re
import random


In [5]:
conn = sqlite3.connect('/content/drive/MyDrive/eng_subtitles_database.db')
query = "SELECT * FROM zipfiles"
df = pd.read_sql_query(query, conn)

num_rows_to_process = int(len(df) * 0.1) #15%
random.seed(42)
selected_rows = df.sample(n=num_rows_to_process)

selected_rows.head()

,num,name,content
73447,9483507,the.rookie.feds.s01.e19.burn.run.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00kh\x9aV\xc...
8031,9214995,the.things.we.bloom.(2021).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x11\x90\x...
5099,9203423,the.dunes.(2021).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00h\x8d\x99V...
24239,9277942,the.ateam.s04.e07.body.slam.(1985).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xdd\xb0\x...
41170,9349301,mythical.beasts.s01.e05.blood.for.the.snake.go...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xd8\xbe\x...


In [6]:
def preprocess_subtitle(subtitle_content):
    cleaned_text = ""

    # Unzip the content
    with zipfile.ZipFile(io.BytesIO(subtitle_content), 'r') as zip_ref:

        extracted_file = zip_ref.extract(zip_ref.namelist()[0])

    # Decode
    with open(extracted_file, 'r', encoding='latin-1') as f:
        for line in f:
            # Remove timestamp
            cleaned_line = re.sub(r'\d+:\d+:\d+,\d+ --> \d+:\d+:\d+,\d+\n', '', line)
            cleaned_text += cleaned_line

    subtitle_text_lower = cleaned_text.lower()

    # Remove special characters
    cleaned_text = re.sub("Watch any video online with Open-SUBTITLES|Free Browser extension: osdb.link/ext","", subtitle_text_lower)
    cleaned_text = re.sub("Please rate this subtitle at www.osdb.link/agwma|Help other users to choose the best subtitles","",cleaned_text)
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', cleaned_text)
    cleaned_text = re.sub(r'[\n\t]', ' ', cleaned_text)
    return cleaned_text

def chunk_subtitle(subtitle_content, chunk_size, overlap_size):
    chunks = []
    chunk_start = 0
    while chunk_start < len(subtitle_content):
        chunk_end = min(chunk_start + chunk_size, len(subtitle_content))
        chunks.append(subtitle_content[chunk_start:chunk_end])
        chunk_start += (chunk_size - overlap_size)
    return chunks

In [7]:
#filename-subtitle pairs
subtitle_map = {}

for index, row in selected_rows.iterrows():
    filename = row['name']
    cleaned_filename = filename.replace('eng.1cd', '').replace('.', ' ')
    subtitle_content = row['content']
    cleaned_subtitle = preprocess_subtitle(subtitle_content)
    # Chunk
    chunks = chunk_subtitle(cleaned_subtitle, chunk_size=500, overlap_size=100)
    subtitle_map[cleaned_filename] = chunks

In [9]:
import csv
csv_file = 'data.csv'

# Open the CSV file in write mode
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Content'])
    for key, value in subtitle_map.items():
        writer.writerow([key, value])

print('CSV file created successfully!')

CSV file created successfully!


In [10]:
df=pd.read_csv('/content/data.csv')

In [11]:
df.head()

,Name,Content
0,the rookie feds s01 e19 burn run (2023),['1 ipreviously on rookie fedsi 2 i think you...
1,the things we bloom (2021),['1 support us and become vip member to remov...
2,the dunes (2021),['1 watch any video online with opensubtitles ...
3,the ateam s04 e07 body slam (1985),['1 all right he gonna put him down now i can ...
4,mythical beasts s01 e05 blood for the snake go...,['1 hissing 2 narrator the feathered serpen...


In [12]:
df['Content'][3]

"['1 all right he gonna put him down now i can feel it  2 crowd cheering  3 ba hey man  4 he didnt save my life i saved his life  5 is there such a thing as a fiveman tag team  6 bring em on daddy  7 shouting  8 no you took the shell  9 hulkamania is gonna run wild  10 no way out this time smith  11 iface now were cookingi  12 oh i love it  13 imale narrator in 1972 a crack commando uniti  14 iwas sent to prison by a military courti  15 ifor a crime they didnt commiti  16 ithese men promptly escape', 'nt to prison by a military courti  15 ifor a crime they didnt commiti  16 ithese men promptly escapedi  17 ifrom a maximumsecurity stockadei  18 ito the los angeles undergroundi  19 itoday still wanted by the governmenti  20 ithey survive as soldiers of fortunei  21 iif you have a problem if no one else can helpi  22 iand if you can find themi  23 imaybe you can hire the ateami  24 gun firing  25 crickets chirping  26 iring announcer and big john studd slams hulk hogan on the mati  27 ith

In [13]:
def preprocess(text):

     cleaned_text = re.sub("use the free code joinnow at  wwwplayshipseu","", text)
     cleaned_text = re.sub("watch any video online with opensubtitles free browser extension osdblinkext","",cleaned_text)
     # Remove the ID
     cleaned_text = re.sub(r'\d+\r\n', ' ', cleaned_text)
     cleaned_text = re.sub(r'<[^>]*>', '', cleaned_text)
     cleaned_text = re.sub(r'[^a-zA-Z0-9 ]+', ' ', cleaned_text)
     cleaned_text = cleaned_text.lower()

     return cleaned_text

In [14]:
df['content_clean'] = df['Content'].apply(preprocess)

In [15]:
df.to_csv('Avideo_subtitles.csv')

In [16]:
df['combined_text'] = df['Name'] + ' ' + df['content_clean']

In [5]:
df=pd.read_csv('Avideo_subtitles.csv')

In [4]:
df.head()

,Unnamed: 0,Name,Content,content_clean
0,0,american dad s01 e17 rough trade (2006),['1 good morning usa 2 i got a feeling tha...,good morning usa i got a feeling that i...
1,1,sister sister s04 e13 little man date (1997),['1 well another saturday night 2 and were of...,well another saturday night and were off t...
2,2,love now episode 1 16 (),['1 apiopensubtitlesorg is deprecated please i...,apiopensubtitlesorg is deprecated please imp...
3,3,the sleep experiment (2022),['1 watch any video online with opensubtitles ...,stop take off the bag and face the wind...
4,4,murdoch mysteries s09 e17 from buffalo with lo...,['1 use the free code joinnow at wwwplayships...,indistinct conversations midtempo piano m...


In [6]:
df['combined_text']=df['Name']+' '+df['content_clean']

In [7]:
chunk_size = 1000
chunks = [df[i:i+chunk_size] for i in range(0, len(df), chunk_size)]

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

for chunk in chunks:
    text_column = chunk['combined_text']
    vectors = vectorizer.fit_transform(text_column)

In [9]:
import torch

In [10]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
def bert_vectorize(text):
    # Tokenize text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings from BERT model output
    embeddings = outputs.last_hidden_state.mean(dim=1)  
    return embeddings.numpy()


In [11]:
# Vectorize each chunk
for chunk in chunks:
    text_column = chunk['combined_text']
    embeddings = text_column.apply(bert_vectorize)


In [44]:
# Connect to ChromaDB
client = chromadb.PersistentClient()
collection = client.create_collection(name="subtitles", metadata={"hnsw:space": "cosine"})


TypeError: Collection.add() missing 1 required positional argument: 'ids'

In [54]:
print(embeddings)

7000    [[-0.22828665, 0.10737817, 0.93315804, -0.3094...
7001    [[-0.35071284, 0.36272472, 0.5276593, -0.16588...
7002    [[-0.09392768, 0.2070764, 0.3695174, -0.036049...
7003    [[-0.27933416, 0.17398375, 0.79185164, -0.2466...
7004    [[-0.17893156, 0.06789591, 0.5646217, 0.084793...
                              ...                        
7790    [[-0.10816643, 0.056510597, 0.3465413, -0.1412...
7791    [[-0.34774515, -0.010042042, 0.438911, -0.0483...
7792    [[-0.40614253, 0.3010406, 0.51430047, -0.22900...
7793    [[-0.15506384, 0.13409151, 0.29507613, -0.0746...
7794    [[-0.31907433, 0.15267548, 0.49797863, -0.1297...
Name: combined_text, Length: 795, dtype: object


In [56]:
for i in range(7000, 7795): 
    subtitle_name = df.loc[i, 'Name']  
    subtitle_embedding = embeddings[i] 
    subtitle_id = str(i)  
    collection.add(documents=subtitle_name, embeddings=subtitle_embedding, ids=subtitle_id)

In [43]:
# client.delete_collection('subtitles')

In [59]:
query_embeddings = bert_vectorize("cars")
k = 5  
results = collection.query(query_embeddings=[query_embeddings], n_results=k)

for result in results:
    print(result)


ValueError: Expected each embedding in the embeddings to be a list, got [array([[ 3.00807208e-01,  7.22174272e-02, -5.40377833e-02,
         1.43974826e-01, -5.45000024e-02, -1.45927772e-01,
         1.37229785e-01, -5.20829372e-02, -1.35657415e-01,
        -9.78249777e-03,  1.82801887e-01,  4.90129925e-02,
         2.85392683e-02, -7.69456709e-03, -3.92446488e-01,
        -2.20101953e-01, -1.04609430e-01,  1.38398498e-01,
        -1.92499701e-02,  8.07900131e-02,  2.71441370e-01,
        -3.10674254e-02,  2.81607717e-01, -1.82386920e-01,
        -2.45416164e-03,  3.09585363e-01, -1.30774722e-01,
        -6.54224530e-02, -2.29234576e-01, -2.46564627e-01,
         4.39392542e-03, -5.85095771e-02,  6.71626255e-02,
         6.58630192e-01,  1.93724334e-02, -3.07913929e-01,
         3.44403721e-02, -2.62571096e-01, -7.84052610e-02,
        -1.18868547e-02,  4.22666483e-02,  7.46082738e-02,
         8.90171528e-03, -2.90871084e-01,  7.57003725e-02,
         1.22824572e-01, -5.89056425e-02, 

In [33]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_names = [row[0] for row in cursor.fetchall()]

In [34]:
for table_name in table_names:
    print(table_name)

migrations
embeddings_queue
collection_metadata
segments
segment_metadata
tenants
databases
collections
embeddings
embedding_metadata
max_seq_id
embedding_fulltext_search
embedding_fulltext_search_data
embedding_fulltext_search_idx
embedding_fulltext_search_content
embedding_fulltext_search_docsize
embedding_fulltext_search_config
